In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Carregamento dos arquivos
dados_estoque = pd.read_excel("Fontes_Dados_Detalhadas.xlsx")
compras_online = pd.read_excel("Compras_Roupas_Online_2024.xlsx")
clima = pd.read_excel("Clima_CentroOeste_2024.xlsx")

# Conversão de datas
compras_online['Data'] = pd.to_datetime(compras_online['Data'])
compras_online['Mes'] = compras_online['Data'].dt.month

# =========================================
# 1. ANÁLISE GERAL
# =========================================
print("\n🔍 Resumo dos Dados de Estoque:")
print(dados_estoque.describe(include='all'))

print("\n🔍 Resumo das Compras Online:")
print(compras_online.describe(include='all'))

# =========================================
# 2. VENDAS POR CATEGORIA
# =========================================
vendas_categoria = compras_online.groupby('Categoria')['Valor'].sum().sort_values(ascending=False)
vendas_categoria.plot(kind='bar', figsize=(10, 5), title="Vendas por Categoria")
plt.ylabel("Valor Total (R$)")
plt.tight_layout()
plt.show()

# =========================================
# 3. VENDAS POR MÊS
# =========================================
vendas_mes = compras_online.groupby('Mes')['Valor'].sum()
vendas_mes.plot(kind='line', marker='o', title="Vendas Totais por Mês", figsize=(10, 4))
plt.ylabel("R$")
plt.grid(True)
plt.show()

# =========================================
# 4. CORRELAÇÃO COM CLIMA
# =========================================
vendas_por_mes = compras_online.groupby('Mes')['Valor'].sum().reset_index()
vendas_por_mes.columns = ['Mes', 'Vendas']

clima_agg = clima.groupby('Mês').agg({'Temperatura Média (°C)': 'mean'}).reset_index()
df_clima_vendas = pd.merge(vendas_por_mes, clima_agg, left_on='Mes', right_on='Mês')

correlacao = df_clima_vendas['Vendas'].corr(df_clima_vendas['Temperatura Média (°C)'])
print(f"\n🌡️ Correlação Temperatura x Vendas: {correlacao:.2f}")

sns.scatterplot(data=df_clima_vendas, x='Temperatura Média (°C)', y='Vendas')
sns.regplot(data=df_clima_vendas, x='Temperatura Média (°C)', y='Vendas', scatter=False, color='red')
plt.title("Temperatura Média x Vendas")
plt.show()

# =========================================
# 5. TICKET MÉDIO
# =========================================
ticket_medio = compras_online['Valor'].mean()
print(f"\n💸 Ticket médio das compras: R$ {ticket_medio:.2f}")

# =========================================
# 6. VENDAS POR REGIÃO
# =========================================
if 'Região' in compras_online.columns:
    vendas_regiao = compras_online.groupby('Região')['Valor'].sum().sort_values(ascending=False)
    print("\n📍 Vendas por Região:")
    print(vendas_regiao)

    vendas_regiao.plot(kind='bar', figsize=(8, 4), title="Vendas por Região", color='skyblue')
    plt.ylabel("R$")
    plt.tight_layout()
    plt.show()

# =========================================
# 7. ANÁLISE POR TAMANHO E COR
# =========================================
if 'Tamanho' in dados_estoque.columns:
    vendas_tamanho = dados_estoque.groupby('Tamanho')['Vendas Mensais'].sum().sort_values(ascending=False)
    print("\n📏 Vendas por Tamanho:")
    print(vendas_tamanho)

    vendas_tamanho.plot(kind='bar', title='Vendas por Tamanho', color='orange')
    plt.show()

if 'Cor' in dados_estoque.columns:
    vendas_cor = dados_estoque.groupby('Cor')['Vendas Mensais'].sum().sort_values(ascending=False).head(10)
    print("\n🎨 Top 10 Cores Mais Vendidas:")
    print(vendas_cor)

    vendas_cor.plot(kind='bar', title='Top 10 Cores Vendidas', color='purple')
    plt.show()

# =========================================
# 8. EXCESSO / FALTA DE ESTOQUE
# =========================================
dados_estoque['Sobra/Falta'] = dados_estoque['Estoque Atual'] - dados_estoque['Vendas Mensais']

excesso_estoque = dados_estoque[dados_estoque['Sobra/Falta'] > 30]
falta_estoque = dados_estoque[dados_estoque['Sobra/Falta'] < 0]

print(f"\n📦 Itens com EXCESSO de estoque (>30): {len(excesso_estoque)}")
print(f"🚫 Itens com FALTA de estoque (<0): {len(falta_estoque)}")

print("\nExemplo de excesso:")
print(excesso_estoque[['Produto', 'Estoque Atual', 'Vendas Mensais', 'Sobra/Falta']].head())

print("\nExemplo de falta:")
print(falta_estoque[['Produto', 'Estoque Atual', 'Vendas Mensais', 'Sobra/Falta']].head())
